# Model Analysis

The script guides through the counting of reactions in each compartment and the generation of the Venn diagramm for reaction overlap of the two K. phaffii models iMT1026, iRY1243 and the O. polymorpha model iUL909.

In [8]:
import pandas as pd
import numpy as np
import re
import os
from datetime import date

# loading matplotlib, a library for visualization
import matplotlib.pyplot as plt
%matplotlib inline

# loading cobrapy, a library dedicated to the analysis of genome scale metabolic models
from cobra.io import read_sbml_model
from cobra import Reaction

Today = date.today().strftime('%y%m%d')

In [9]:
def CoSub(Flux, C1, C2, R):
    F1 = Flux / (C1*(1+1/R))
    F2 = F1*C1 / (R*C2)
    return F1, F2

def countCinFormula(formula):
    return int(re.search(r'C(.*)H',formula).group(1)) if re.search(r'C(.*)H',formula).group(1) else int(1)

def set_ObjiUma(model, Product):
        model.add_boundary(model.metabolites.get_by_id(Product), type='demand')
        model.objective = 'DM_{}'.format(Product)
#         sol = md.optimize()
        return model

def set_SubiUma(model, Sub, Flux, Ratio):
    Catoms = [countCinFormula(model.metabolites.get_by_id(subi).formula) for subi in Sub]
    Cup = Flux/np.sum(Catoms)
#         print(sub,Cup)
    model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
    if len(Sub)<2:
#         print(Sub, Cup)
        model.reactions.get_by_id('EX_{}'.format(Sub[0])).lower_bound = -Cup
    else:
#         print('else ', len(sub))
        C1 = Catoms[0]
        C2 = Catoms[1]
        F1,F2 = CoSub(Flux, C1, C2, Ratio)
#         print(F1, F2)
        for subi, fluxi in zip(Sub, [F1,F2]):
            model.reactions.get_by_id('EX_{}'.format(subi)).lower_bound = -fluxi
#         print(md.slim_optimize())
    return model

In [96]:
# generating cobra variable from SBML/xml file
# assuming the base path is in 'Simulation'
iUma_File = os.path.join('..','model','iUma22.xml') #'iUma22.xml' # os.path.join('..','iUL_versions','iUL959_112718.xml')
model = read_sbml_model(iUma_File)

## Adding missing reactions

In [97]:
# Mannitol production
# https://biocyc.org/META/NEW-IMAGE?type=PATHWAY&object=PWY-3881
ID = 'PseudoMan6P2Mnl'
if ID not in model.reactions:
    reaction = Reaction(ID)
    reaction.name = 'Pseudoreaction from Mannose-6-P to Mannitol'
    reaction.lower_bound = 0.
    reaction.upper_bound = 1000.
#     reaction.annotation = {'biocyc':'MANNPISOM-RXN', 'brenda': '5.3.1.8'}
    model.add_reactions([reaction])
    model.reactions.get_by_id(ID).build_reaction_from_string('MANNOSE-6P_c + nadph_c + h_c + h2o_c --> mnl_c + nadp_c + pi_c')

    print(model.reactions.get_by_id(ID).id)
    print(model.reactions.get_by_id(ID).reaction)
    print(model.reactions.get_by_id(ID).check_mass_balance())
    
model.add_boundary(model.metabolites.get_by_id('2obut_e'), type="exchange", lb=0)
model.add_boundary(model.metabolites.get_by_id('for_e'), type="exchange", lb=0)

# Isocitrate lyase
# https://www.genome.jp/dbget-bin/www_bget?uma:UMAG_01892+uma:UMAG_04285
ID = 'ICL'
if ID not in model.reactions:
    reaction = Reaction(ID)
    reaction.name = 'Isocitrate lyase'
    reaction.lower_bound = 0.
    reaction.upper_bound = 1000.
#     reaction.annotation = {'biocyc':'MANNPISOM-RXN', 'brenda': '5.3.1.8'}
    model.add_reactions([reaction])
    model.reactions.get_by_id(ID).build_reaction_from_string('icit_c --> succ_c + glx_c')

    print(model.reactions.get_by_id(ID).id)
    print(model.reactions.get_by_id(ID).reaction)
    print(model.reactions.get_by_id(ID).check_mass_balance())
    

PseudoMan6P2Mnl
MANNOSE-6P_c + h2o_c + h_c + nadph_c --> mnl_c + nadp_c + pi_c
{}
ICL
icit_c --> glx_c + succ_c
{}


In [ ]:
# Searching for metabolite IDs
CheckMet = 'lycer'
[print('{}:\t{}:\t{}:\t{}\n'.format(MetID.name, MetID.id, MetID.formula, MetID.annotation)) for MetID in model.metabolites.query(CheckMet, 'name')];

In [101]:
ProList = ['itacon_c','succ_c','mal__L_c', 'eryth_c', 'mnl_c','hex2manerythol_c','UA_A_c', 'CPD-15924_c'] # hexACP_c -> UA_A_c, CPD-13014_c: tributyrin
SubList = [['ac_e'], ['for_e'], ['2obut_e'], ['glc__D_e'], ['glyc_e'], ['xyl__D_e'], ['glc__D_e', 'for_e'], ['glc__D_e', 'ac_e']] #['ac_e'], ['for_e'], 
SubProCom = [(x,y) for x in SubList for y in ProList]

In [102]:
Solution_df = pd.DataFrame(columns=['Substrate', 'Product', 'Yield'])
for SubProi in SubProCom:
    Substrate_lst = SubProi[0]
    Product_str = SubProi[1]
    CnumPro = countCinFormula(model.metabolites.get_by_id(Product_str).formula)
    F = 100 # C-mole/gDW/h
    R = 5
    with model as md:
        md = set_SubiUma(md, Substrate_lst, F, R)
        md = set_ObjiUma(md, Product_str)
        sol = md.optimize()
        Yield = sol.fluxes['DM_{}'.format(Product_str)]*CnumPro/F
        Solution_df = Solution_df.append({'Substrate': str(Substrate_lst), 'Product': Product_str, 'Yield': round(Yield,2)}, ignore_index=True)
        mybnd = [md.reactions.get_by_id('EX_{}'.format(subi)).lower_bound for subi in Substrate_lst]
        print('{} production on {}: {:.2f}'.format(Product_str, Substrate_lst, Yield))

C:\Users\liebalu\.conda\envs\iambjudas\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


itacon_c production on ['ac_e']: 0.00
succ_c production on ['ac_e']: 0.00
mal__L_c production on ['ac_e']: 0.00
eryth_c production on ['ac_e']: 0.00
mnl_c production on ['ac_e']: 0.00
hex2manerythol_c production on ['ac_e']: 0.00
UA_A_c production on ['ac_e']: 0.00
CPD-15924_c production on ['ac_e']: 0.00
itacon_c production on ['for_e']: 0.00
succ_c production on ['for_e']: 0.00
mal__L_c production on ['for_e']: 0.00
eryth_c production on ['for_e']: 0.00
mnl_c production on ['for_e']: 0.00
hex2manerythol_c production on ['for_e']: -0.00
UA_A_c production on ['for_e']: 0.00
CPD-15924_c production on ['for_e']: 0.00
itacon_c production on ['2obut_e']: 0.83
succ_c production on ['2obut_e']: 1.00
mal__L_c production on ['2obut_e']: 1.00
eryth_c production on ['2obut_e']: 0.32
mnl_c production on ['2obut_e']: 0.37
hex2manerythol_c production on ['2obut_e']: 0.39
UA_A_c production on ['2obut_e']: 0.46
CPD-15924_c production on ['2obut_e']: 0.46
itacon_c production on ['glc__D_e']: 1.00
succ

In [103]:
Solution_Tab = pd.pivot(Solution_df, values='Yield', index='Substrate', columns='Product')
Solution_Tab['Mean-Substrates'] = round(Solution_Tab.mean(axis=1), 2)
Mean = Solution_Tab.mean()
Mean.name='Mean-Products'
Solution_Tab = round(Solution_Tab.append(Mean), 2)
Solution_Tab

Product,CPD-15924_c,UA_A_c,eryth_c,hex2manerythol_c,itacon_c,mal__L_c,mnl_c,succ_c,Mean-Substrates
Substrate,,,,,,,,,
['2obut_e'],0.46,0.46,0.32,0.39,0.83,1.00,0.37,1.00,0.60
['ac_e'],0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
['for_e'],0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00
"['glc__D_e', 'ac_e']",0.70,0.80,0.73,0.83,1.00,1.00,0.82,1.00,0.86
"['glc__D_e', 'for_e']",0.61,0.69,0.73,0.72,1.00,1.00,0.77,1.00,0.82
['glc__D_e'],0.73,0.83,0.88,0.86,1.00,1.00,0.92,1.00,0.90
['glyc_e'],0.86,0.97,0.88,0.99,1.00,1.00,0.99,1.00,0.96
['xyl__D_e'],0.73,0.83,0.79,0.86,1.00,1.00,0.89,1.00,0.89
Mean-Products,0.51,0.57,0.54,0.58,0.73,0.75,0.60,0.75,0.63


In [ ]:
Solution_Tab.to_excel('{}_SubProdTable.xlsx'.format(Today))

In [ ]:
with model as model:
    model.reactions.get_by_id('EX_glc__D_e').lower_bound = -100/6
    model.reactions.get_by_id('EX_for_e').lower_bound = -0
    model = set_ObjiUma(model, 'succ_c')
#     model.objective = 'EX_succ_e'
    print(model.summary())

In [ ]:
with model as model:
    model.reactions.get_by_id('EX_glc__D_e').lower_bound = -100/6
    model.reactions.get_by_id('EX_for_e').lower_bound = -0
    model = set_ObjiUma(model, 'succ_c')
#     model.objective = 'EX_succ_e'
    print(model.summary())

In [104]:
ratio = np.linspace(8,2,2)
mySol = list()
for iratio in ratio:
    glc_up, ace_up = CoSub(100,6,2,iratio)
    print('glc:{}, ace:{}'.format(glc_up, ace_up))
    with model as model:
        model.reactions.get_by_id('EX_glc__D_e').lower_bound = -glc_up
        model.reactions.get_by_id('EX_ac_e').lower_bound = -ace_up
        # model = set_ObjiUma(model, 'succ_c')
    #     model.objective = 'EX_succ_e'
        print(model.summary())
        mySol.append(model.optimize().fluxes)
    print(glc_up, ace_up)
    

glc:14.814814814814815, ace:5.555555555555555
Objective
1.0 BIOMASS_REACTION = 1.4545570348519654

Uptake
------
Metabolite    Reaction   Flux  C-Number C-Flux
      ac_e     EX_ac_e  5.556         2 11.11%
  glc__D_e EX_glc__D_e  14.81         6 88.89%
     h2o_e    EX_h2o_e  41.09         0  0.00%
     nh4_e    EX_nh4_e   6.85         0  0.00%
      o2_e     EX_o2_e 0.3642         0  0.00%
     so4_e    EX_so4_e 0.1193         0  0.00%

Secretion
---------
Metabolite    Reaction   Flux  C-Number C-Flux
     adn_e    EX_adn_e -20.57        10 84.49%
     for_e    EX_for_e -14.61         1  6.00%
     fum_e    EX_fum_e -0.256         4  0.42%
  mal__L_e EX_mal__L_e -5.528         4  9.08%
      pi_e     EX_pi_e -60.96         0  0.00%

14.814814814814815 5.555555555555555
glc:11.11111111111111, ace:16.666666666666664
Objective
1.0 BIOMASS_REACTION = 1.0868221286208457

Uptake
------
Metabolite    Reaction    Flux  C-Number C-Flux
      ac_e     EX_ac_e   12.79         2 27.73%
  glc__D

In [94]:
mySol[0]['TRANS-RXNTFS-1-ACET1223']
[print('{},\t{},\t{}'.format(rct.id, mySol[0][rct.id], rct.reaction)) for rct in model.metabolites.get_by_id('icit_c').reactions if mySol[0][rct.id]>0];

In [80]:
[print(rct.id, rct.reaction) for rct in model.metabolites.get_by_id('ac_c').reactions];

TRANS-RXNTFS-8-ACET1050 ac_e --> ac_c
LO lac__L_c + o2_c --> ac_c + co2_c + h2o_c
ACSp ac_c + atp_c + coa_c --> accoa_c + amp_c + ppi_c
ACKrm ac_c + atp_c <=> actp_c + adp_c
ALDD2xm acald_c + h2o_c + nad_c --> ac_c + 2.0 h_c + nadh_c
CYSS_1 acser_c + h2s_c <=> ac_c + cys__L_c + h_c
SLCYSS acser_c + tsul_c <=> ac_c + h_c + slcys_c
ALDD2ym acald_c + h2o_c + nadp_c --> ac_c + 2.0 h_c + nadph_c
AHSERL2 achms_c + h2s_c --> ac_c + h_c + hcys__L_c
TRANS-RXNTFS-1-ACET1223 ac_e --> ac_c
NAGDA acgam6p_c + h2o_c --> ac_c + gam6p_c
TRANS-RXNTFS-58-ACET34 ac_e + h_e --> ac_c + h_c
ACETATE--COA-LIGASE-ADP-FORMING-RXN ac_c + atp_c + coa_c <=> accoa_c + adp_c + pi_c
RXN-14728 ad_c + h2o_c --> ac_c + nh4_c
METB1 achms_c + cys__L_c --> ac_c + cyst__L_c + h_c
TRANS-RXNTFS-7-ACET1018 ac_e --> ac_c


In [68]:
model.metabolites.query('aceta','name')

[<Metabolite id3acald_c at 0x2249e351bb0>,
 <Metabolite pac_c at 0x2249e3c7970>,
 <Metabolite fpram_c at 0x2249e3d18b0>,
 <Metabolite CPD-12708_c at 0x2249e3ec9d0>,
 <Metabolite 4mlacac_c at 0x2249e3ece80>,
 <Metabolite 4fumacac_c at 0x2249e3eceb0>,
 <Metabolite acald_c at 0x2249e418df0>,
 <Metabolite oaa_c at 0x2249e418fa0>,
 <Metabolite oaa_m at 0x2249e418d00>,
 <Metabolite ind3ac_c at 0x2249e425ca0>,
 <Metabolite ad_c at 0x2249e425ee0>,
 <Metabolite acac_c at 0x2249e4416d0>,
 <Metabolite ac_e at 0x2249e4419d0>,
 <Metabolite ac_c at 0x2249e441a00>,
 <Metabolite CPD-12709_e at 0x2249e441b80>,
 <Metabolite CPD-12709_c at 0x2249e441bb0>]

In [95]:
model.reactions.get_by_id('ICL')

KeyError: 'ICL'